In [185]:
import pandas as pd
import json
import re
import plotly.express as px
import numpy as np
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

### RQ2.4
varying the values (context) of the parameters in the given clause

In [186]:
current_result = "4_shared_scenarios_results"

In [187]:
file_path = f'res/outcome/similarities/{current_result}.csv'
df = pd.read_csv(file_path, sep=';')

In [188]:
df.drop(columns=['Desired When DNF',
       'Shared When DNF', 'Penality When', 'Global Similarity When',
       'Final Similarity When',"Desired Then DNF","Shared Then DNF","Penality Then","Global Similarity Then","Final Similarity Then"], inplace=True)
df

,Scenario,Desired Given DNF,Shared Given DNF,Penality Given,Global Similarity Given,Final Similarity Given,Scenario Similarity,Action
0,Candidate_1,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 7 AND win...,0.0,1.000000,1.000000,1.000000,behavior_1
1,Candidate_1585,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 100 AND satellite_count >= 7 AND wi...,0.0,0.966667,0.966667,0.988889,behavior_1585
2,Candidate_145,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 0 AND satellite_count >= 7 AND wind...,0.0,0.966667,0.966667,0.988889,behavior_145
3,Candidate_109,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 8 AND win...,0.0,0.958333,0.958333,0.986111,behavior_109
4,Candidate_37,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 8 AND win...,0.0,0.958333,0.958333,0.986111,behavior_37
...,...,...,...,...,...,...,...,...
1723,Candidate_1007,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,0.0,0.000000,0.000000,0.666667,behavior_1007
1724,Candidate_782,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 100 AND satellite_count >= 15 AND w...,0.0,0.000000,0.000000,0.666667,behavior_782
1725,Candidate_1006,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,0.0,0.000000,0.000000,0.666667,behavior_1006
1726,Candidate_1005,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,0.0,0.000000,0.000000,0.666667,behavior_1005


In [189]:
def extract_clauses(expression):
    # Primeiro, dividir a expressão usando 'AND' como delimitador
    clauses = expression.split(' AND ')
    
    # Regex para capturar o padrão: variável, operador, e valor, incluindo == e !=
    pattern = r'(\w+)\s*(>=|<=|>|<|=|==|!=)\s*([\d.]+)'
    
    # Extrair e converter as cláusulas em um dicionário
    result = {}
    for clause in clauses:
        match = re.match(pattern, clause.strip())
        if match:
            var, op, val = match.groups()
            val = float(val)
            result[var] = (var, op, val)
    
    return result

def create_pairs(row):
    clauses_1 = extract_clauses(row['Desired Given DNF'])
    clauses_2 = extract_clauses(row['Shared Given DNF'])
    
    # Obter o conjunto de todas as variáveis únicas das duas colunas
    all_vars = set(clauses_1.keys()).union(set(clauses_2.keys()))
    
    # Criar o vetor de duplas
    pairs = [(clauses_1.get(var, ('', '', '')), clauses_2.get(var, ('', '', ''))) for var in all_vars]
    
    return pairs


# Aplicar a função para criar o vetor de duplas para cada linha do DataFrame
df['Pairs'] = df.apply(create_pairs, axis=1)

# Exibir o DataFrame com os vetores de duplas
df[['Desired Given DNF', 'Shared Given DNF', 'Pairs']]


,Desired Given DNF,Shared Given DNF,Pairs
0,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 7 AND win...,"[((battery, >=, 10.0), (battery, >=, 10.0)), (..."
1,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 100 AND satellite_count >= 7 AND wi...,"[((battery, >=, 10.0), (battery, <=, 100.0)), ..."
2,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 0 AND satellite_count >= 7 AND wind...,"[((battery, >=, 10.0), (battery, >=, 0.0)), ((..."
3,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 8 AND win...,"[((battery, >=, 10.0), (battery, >=, 10.0)), (..."
4,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 10 AND satellite_count >= 8 AND win...,"[((battery, >=, 10.0), (battery, >=, 10.0)), (..."
...,...,...,...
1723,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,"[((battery, >=, 10.0), (battery, <=, 10.0)), (..."
1724,battery >= 10 AND satellite_count >= 7 AND win...,battery >= 100 AND satellite_count >= 15 AND w...,"[((battery, >=, 10.0), (battery, >=, 100.0)), ..."
1725,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,"[((battery, >=, 10.0), (battery, <=, 10.0)), (..."
1726,battery >= 10 AND satellite_count >= 7 AND win...,battery <= 10 AND satellite_count >= 15 AND wi...,"[((battery, >=, 10.0), (battery, <=, 10.0)), (..."


In [190]:
df[["Pairs"]].iloc[0]

Pairs    [((battery, >=, 10.0), (battery, >=, 10.0)), (...
Name: 0, dtype: object

In [191]:
# Carregando os limites das variáveis a partir do JSON
with open('res/variables_config.json', 'r') as file:
    variable_configs = json.load(file)

# Criar um dicionário com os limites de cada variável
limits = {var['name']: {'min': var['min_value'], 'max': var['max_value']} for var in variable_configs}

# Função para calcular o intervalo baseado no operador
def calculate_interval(var, op, val):
    min_limit, max_limit = limits[var]['min'], limits[var]['max']
    
    if op == '>=':
        return (val, max_limit)
    elif op == '<=':
        return (min_limit, val)
    elif op == '>':
        return (val, max_limit)
    elif op == '<':
        return (min_limit, val)
    elif op == '==':
        return (val, val)
    elif op == '!=':
        return (min_limit, max_limit)  # assume o escopo completo
    else:
        raise ValueError(f"Operador '{op}' não reconhecido.")

# Função para calcular o erro característico entre duas tuplas
def calculate_scope_error(pair):
    tuple1, tuple2 = pair
    
    if tuple1[0] != tuple2[0]:  # Verifica se as variáveis são diferentes
        return f"Erro: Variáveis diferentes {tuple1[0]} e {tuple2[0]}"
    
    var = tuple1[0]
    interval1 = calculate_interval(*tuple1)
    interval2 = calculate_interval(*tuple2)
    
    # Calculando o erro característico
    scope_error = (abs(interval1[0] - interval2[0]), abs(interval1[1] - interval2[1]))
    return scope_error

# Exemplo de DataFrame com pares
# data = {
#     'Pairs': [[(('battery', '>=', 10.0), ('battery', '<=', 50.0))],
#               [(('satellite_count', '>=', 5), ('satellite_count', '<=', 10))],
#               [(('wind_speed', '>', 20.0), ('wind_speed', '<', 30.0))]]
# }

# df = pd.DataFrame(data)

# Aplicar a função para calcular o erro característico para cada par de tuplas em 'Pairs'
df['Scope Error by var'] = df['Pairs'].apply(lambda pairs: [calculate_scope_error(pair) for pair in pairs])

# Exibir o DataFrame com os erros característicos
df[['Pairs', 'Scope Error by var']].head()

,Pairs,Scope Error by var
0,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (0.0, 0)]"
1,"[((battery, >=, 10.0), (battery, <=, 100.0)), ...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]"
2,"[((battery, >=, 10.0), (battery, >=, 0.0)), ((...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]"
3,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]"
4,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]"


In [192]:
# Função para normalizar o erro com base nos limites da variável
def normalize_error(error, variable):
    if variable in limits:
        min_value = limits[variable]['min']
        max_value = limits[variable]['max']
        # Normaliza o erro usando o intervalo da variável
        return error / (max_value - min_value) if max_value - min_value != 0 else 0
    else:
        return error  # Se a variável não estiver no JSON, retorna o erro sem normalização

# Função para normalizar os erros no 'Scope Error by var' usando a variável identificada nos pares
def normalize_scope_error(pairs, errors):
    normalized_errors = []
    for i, pair in enumerate(pairs):
        variable = pair[0][0]  # Identifica a variável a partir do primeiro elemento da tupla do par
        error = errors[i]
        # Normaliza cada erro dentro da lista de erros associados ao par de variáveis
        normalized_error = [normalize_error(e, variable) for e in error]
        normalized_errors.append(normalized_error)
    return normalized_errors

In [193]:
df['Normalized Scope Error by var'] = df.apply(
    lambda row: normalize_scope_error(row['Pairs'], row['Scope Error by var']), axis=1
)

# Exibir o DataFrame com os erros normalizados
df[['Pairs', 'Scope Error by var', 'Normalized Scope Error by var']].head()

,Pairs,Scope Error by var,Normalized Scope Error by var
0,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]"
1,"[((battery, >=, 10.0), (battery, <=, 100.0)), ...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.1, 0.0], [0.0, 0.0], [0.0, 0.0]]"
2,"[((battery, >=, 10.0), (battery, >=, 0.0)), ((...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.1, 0.0], [0.0, 0.0], [0.0, 0.0]]"
3,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.06666666666666667,..."
4,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.06666666666666667,..."


In [194]:
def sum_normalized_errors(normalized_errors):
    # Flatten a lista de listas e soma todos os valores
    return sum(sum(error) for error in normalized_errors)

# Calcular o erro escalar somando todos os valores normalizados
df['Global Normalized Error Sum'] = df['Normalized Scope Error by var'].apply(sum_normalized_errors)

# Exibir o DataFrame com o valor escalar do erro global normalizado
df[['Pairs', 'Scope Error by var', 'Normalized Scope Error by var', 'Global Normalized Error Sum']]

,Pairs,Scope Error by var,Normalized Scope Error by var,Global Normalized Error Sum
0,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]",0.000000
1,"[((battery, >=, 10.0), (battery, <=, 100.0)), ...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.1, 0.0], [0.0, 0.0], [0.0, 0.0]]",0.100000
2,"[((battery, >=, 10.0), (battery, >=, 0.0)), ((...","[(10.0, 0.0), (0.0, 0.0), (0.0, 0)]","[[0.1, 0.0], [0.0, 0.0], [0.0, 0.0]]",0.100000
3,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.06666666666666667,...",0.066667
4,"[((battery, >=, 10.0), (battery, >=, 10.0)), (...","[(0.0, 0.0), (0.0, 0.0), (1.0, 0)]","[[0.0, 0.0], [0.0, 0.0], [0.06666666666666667,...",0.066667
...,...,...,...,...
1723,"[((battery, >=, 10.0), (battery, <=, 10.0)), (...","[(10.0, 90.0), (270.0, 324.0), (8.0, 0)]","[[0.1, 0.9], [0.75, 0.9], [0.5333333333333333,...",3.183333
1724,"[((battery, >=, 10.0), (battery, >=, 100.0)), ...","[(90.0, 0.0), (0.0, 36.0), (8.0, 0)]","[[0.9, 0.0], [0.0, 0.1], [0.5333333333333333, ...",1.533333
1725,"[((battery, >=, 10.0), (battery, <=, 10.0)), (...","[(10.0, 90.0), (180.0, 324.0), (8.0, 0)]","[[0.1, 0.9], [0.5, 0.9], [0.5333333333333333, ...",2.933333
1726,"[((battery, >=, 10.0), (battery, <=, 10.0)), (...","[(10.0, 90.0), (90.0, 324.0), (8.0, 0)]","[[0.1, 0.9], [0.25, 0.9], [0.5333333333333333,...",2.683333


In [195]:
df[['Global Normalized Error Sum']].describe()

,Global Normalized Error Sum
count,1728.000000
mean,1.716667
std,0.745882
min,0.000000
25%,1.183333
50%,1.700000
75%,2.250000
max,4.000000


In [196]:
df[['Global Normalized Error Sum','Scenario Similarity']]

,Global Normalized Error Sum,Scenario Similarity
0,0.000000,1.000000
1,0.100000,0.988889
2,0.100000,0.988889
3,0.066667,0.986111
4,0.066667,0.986111
...,...,...
1723,3.183333,0.666667
1724,1.533333,0.666667
1725,2.933333,0.666667
1726,2.683333,0.666667


In [197]:
# Criar o scatter plot usando o Plotly
fig = px.scatter(
    df,
    x='Global Normalized Error Sum',
    y='Scenario Similarity',
    title='Accumulated distance from Given vs Scenario Similarity',
    labels={
        'Global Normalized Error Sum': 'Accumulated distance',
        'Scenario Similarity': 'Scenario Similarity'
    },
    template='plotly_white'
)

# Exibir o gráfico
fig.show()

Tentar entender pq no maximo foi 4, como eram 3 variáveis formando 6 [chão, tento], imaginei que no pior dos casos, todos os erros de cada um iria tentar a 1, ou seja [1,1], [1,1], [1,1], logo, somando daria 6. Mas no caso foi 4. Tlz nas condicoes nao trazem situacoes desfavorável para todas as variaveis.